In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from ibm_watson_machine_learning import APIClient
import pandas as pd

In [3]:
url = "https://us-south.ml.cloud.ibm.com"

#create API key here: https://cloud.ibm.com/iam/apikeys
apikey =""


wml_credentials = {
      "apikey": apikey,
      "url": url
}

client = APIClient(wml_credentials)
client.version

'1.0.309'

In [4]:
# Prepare your model archive

In [5]:
try:
    !rmdir model
except:
    True
     
try:
    !rm model.py
except:
    True
try:    
    !rm -r model  
except:
    True
try:    
    !rm model.tar.gz
except:
    True    


rmdir: failed to remove 'model': Directory not empty
rm: cannot remove 'model.py': No such file or directory


In [6]:
%mkdir model

In [7]:
%%writefile model/main.py

from docplex.util.environment import get_environment
import pandas
from six import iteritems
from collections.abc import Mapping
from os.path import join, dirname, basename, splitext, exists
import glob

class _InputDict(dict):
    def __init__(self, directory, names):
        dict.__init__(self)
        self._directory = directory
        for k in names:
            dict.__setitem__(self, k, None)
        file='model_schema.json'
        if self._directory is not None:
            file  = "{0}/".format(self._directory) + file
        self.dtype_schemas = self.get_dtype_schemas( file)
    def __getitem__(self, key):
        if isinstance(key, str):
            item = dict.__getitem__(self, key)
            if item is None:
                file = "{0}.csv".format(key)
                if file in self.dtype_schemas:
                    return self.read_df( key, dtype=self.dtype_schemas[file])
                else:
                    return self.read_df( key)
            else:
                return item
        else:
            raise Exception("Accessing input dict via non string index")
    def read_df(self, key, **kwargs):
        env = get_environment()
        file = "{0}.csv".format(key)
        if self._directory is not None:
            file  = "{0}/".format(self._directory) + file
        with env.get_input_stream(file) as ist:
            params = {'encoding': 'utf8'}
            if kwargs:
                params.update(kwargs)
            df = pandas.read_csv( ist, **params)
            dict.__setitem__(self, key, df)
        return df
    def get_dtype_schemas(self, path):
        dtype_schemas = {}
        if exists(path):
            input_schemas=json.load(open(path))
            if 'input' in input_schemas:
                for input_schema in input_schemas['input']:
                    dtype_schema = {}
                    if 'fields' in input_schema:
                        for input_schema_field in input_schema['fields']:
                            if input_schema_field['type']=='string':
                                dtype_schema[input_schema_field['name']]='str'
                        if len(dtype_schema) > 0:
                            dtype_schemas[input_schema['id']]=dtype_schema
        print(dtype_schemas)
        return dtype_schemas

class _LazyDict(Mapping):
    def __init__(self, *args, **kw):
        self._raw_dict = _InputDict(*args, **kw)

    def __getitem__(self, key):
        return self._raw_dict.__getitem__(key)

    def __iter__(self):
        return iter(self._raw_dict)

    def __len__(self):
        return len(self._raw_dict)

    def read_df(self, key, **kwargs):
        return self._raw_dict.read_df(key, **kwargs)

def get_all_inputs(directory=None):
    '''Utility method to read a list of files and return a tuple with all
    read data frames.
    Returns:
        a map { datasetname: data frame }
    '''

    all_csv = "*.csv"
    g = join(directory, all_csv) if directory else all_csv

    names = [splitext(basename(f))[0] for f in glob.glob(g)]
    result = _LazyDict(directory, names)
    return result

def write_all_outputs(outputs):
    '''Write all dataframes in ``outputs`` as .csv.

    Args:
        outputs: The map of outputs 'outputname' -> 'output df'
    '''
    for (name, df) in iteritems(outputs):
        csv_file = '%s.csv' % name
        print(csv_file)
        with get_environment().get_output_stream(csv_file) as fp:
            if sys.version_info[0] < 3:
                fp.write(df.to_csv(index=False, encoding='utf8'))
            else:
                fp.write(df.to_csv(index=False).encode(encoding='utf8'))
    if len(outputs) == 0:
        print("Warning: no outputs written")

Writing model/main.py


In [8]:
%%writefile -a model/main.py

import pandas as pd
from docplex.mp.model import Model

def continuous_var_series(df, mdl,**kargs):
    return pd.Series(mdl.continuous_var_list(df.index, **kargs), index = df.index)

class CplexSum():
    """Function class that adds a series of dvars into a cplex sum expression.
    To be used as a custom aggregation in a groupby.
    Usage:
        df2 = df1.groupby(['a']).agg({'xDVar':CplexSum(engine.mdl)}).rename(columns={'xDVar':'expr'})

    Sums the dvars in the 'xDVar' column into an expression
    """
    def __init__(self, mdl):
        self.mdl = mdl
    def __call__(self, dvar_series):
        return self.mdl.sum(dvar_series)


def extract_solution(df, extract_dvar_names=None, drop_column_names=None, drop:bool=True):
    df = df.copy()
    """Generalized routine to extract a solution value. 
    Can remove the dvar column from the df to be able to have a clean df for export into scenario."""
    if extract_dvar_names is not None:
        for xDVarName in extract_dvar_names:
            if xDVarName in df.columns:
                df[f'{xDVarName}_Solution'] = [dvar.solution_value for dvar in df[xDVarName]]
                if drop:
                    df = df.drop([xDVarName], axis = 1)
    if drop and drop_column_names is not None:
        for column in drop_column_names:
            if column in df.columns:
                df = df.drop([column], axis = 1)
    return df  


inputs = get_all_inputs()

Plants = inputs['Plants']
Customers = inputs['Customers']
Products = inputs['Products']
ProductionCosts= inputs['ProductionCosts']
Demand= inputs['Demand']
TransportationCosts= inputs['TransportationCosts']
ProductionCapacity= inputs['ProductionCapacity']

MODEL_NAME = 'SupplyChain'
mdl = Model(MODEL_NAME)

ProductionPlan = pd.merge(Products.reset_index(drop=True),Plants.reset_index(drop=True),how="cross").set_index(["ProductName","PlantName"])
ProductionPlan["X_production"]= continuous_var_series(ProductionPlan, mdl, name="X_production", lb = 0)

TransportationPlan = pd.merge(Products.reset_index(drop=True),Plants.reset_index(drop=True),how="cross").merge(Customers.reset_index(drop=True),how="cross").set_index(["ProductName","PlantName","CustomerName"])
TransportationPlan["X_transportation"]= continuous_var_series(TransportationPlan, mdl, name="X_transportation", lb = 0)

Plant_Product_Capacity = pd.merge(ProductionPlan.reset_index(drop=False), ProductionCapacity, on = ["ProductName","PlantName"])
for row in Plant_Product_Capacity.itertuples():
    mdl.add_constraint(row.X_production <= row.Capacity)
    
    
Delivery = TransportationPlan.reset_index(drop = False)
Delivery = Delivery[["ProductName","CustomerName","X_transportation"]].groupby(["ProductName","CustomerName"]).agg(CplexSum(mdl)).reset_index(drop = False).rename(columns = {"X_transportation":"Total_Delivered"})
Delivery = pd.merge(Delivery, Demand.reset_index(drop = True), on = ["ProductName","CustomerName"] )

for row in Delivery.itertuples():
    mdl.add_constraint(row.Total_Delivered >= row.Demand)    
    
    
Shipment = TransportationPlan.reset_index(drop = False)
Shipment = Shipment[["ProductName","PlantName","X_transportation"]].groupby(["ProductName","PlantName"]).agg(CplexSum(mdl)).reset_index(drop = False).rename(columns = {"X_transportation":"Total_Shipped"})
Plant_Shipment = pd.merge(ProductionPlan.reset_index(drop= False), Shipment, on =[ "ProductName","PlantName"])

for row in Plant_Shipment.itertuples():
    mdl.add_constraint(row.Total_Shipped <= row.X_production)
    
    
TotalProductionCosts = pd.merge(ProductionPlan.reset_index(drop = False), ProductionCosts.reset_index(drop = True), on = ["ProductName","PlantName"])
Total_Production_Costs = mdl.sum(TotalProductionCosts.ProductionCost * TotalProductionCosts.X_production)
mdl.add_kpi(Total_Production_Costs   , "Total_Production_Costs")


TotalTransportationCosts = pd.merge(TransportationPlan.reset_index(drop = False), TransportationCosts.reset_index(drop = False), on = ["ProductName","CustomerName","PlantName"])
Total_Transportation_Costs = mdl.sum(TotalTransportationCosts.TransportationCost * TotalTransportationCosts.X_transportation)
mdl.add_kpi(Total_Transportation_Costs   , "Total_Transportation_Costs")


Total_Costs = Total_Transportation_Costs + Total_Production_Costs
mdl.add_kpi(Total_Costs   , "Total_Costs")
mdl.print_information()


mdl.minimize(Total_Costs)
mdl.solve()
mdl.report()



ProductionPlan = extract_solution(Plant_Product_Capacity, extract_dvar_names= ['X_production'] ,drop=False)
TransportationPlan = extract_solution(TransportationPlan, extract_dvar_names= ['X_transportation'] ,drop=False).reset_index(drop = False)
Delivery = extract_solution(Delivery, extract_dvar_names= ['Total_Delivered'] ,drop=False)

outputs={}
outputs['ProductionPlan']=ProductionPlan
outputs['TransportationPlan']=TransportationPlan
outputs['Delivery']=Delivery
    
# Generate output files
write_all_outputs(outputs)

Appending to model/main.py


In [9]:
import tarfile
def reset(tarinfo):
    tarinfo.uid = tarinfo.gid = 0
    tarinfo.uname = tarinfo.gname = "root"
    return tarinfo
tar = tarfile.open("model.tar.gz", "w:gz")
#tar.add("model.py", arcname="model.py", filter=reset)
tar.add("model/main.py", arcname="main.py", filter=reset)
tar.close()

In [10]:
# Upload your model on Watson Machine Learning

In [11]:
# Find the space ID

space_name = 'Deployment_space_DO'

space_id = [x['metadata']['id'] for x in client.spaces.get_details()['resources'] if x['entity']['name'] == space_name][0]

client.set.default_space(space_id)

'SUCCESS'

In [12]:
mnist_metadata = {
    client.repository.ModelMetaNames.NAME: "Supply Chain",
    client.repository.ModelMetaNames.DESCRIPTION: "Model for Supply Chain Optimization",
    client.repository.ModelMetaNames.TYPE: "do-docplex_22.1",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_uid_by_name("do_22.1"),
}

model_details = client.repository.store_model(model='/home/wsuser/work/model.tar.gz', meta_props=mnist_metadata)
#model='/home/wsuser/work/model.tar.gz', 
model_uid = client.repository.get_model_id(model_details)

In [13]:
# Create a deployment

In [14]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "Supply Chain",
    client.deployments.ConfigurationMetaNames.DESCRIPTION: "Supply Chain",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {'name': 'S', 'num_nodes': 1}
}

deployment_details = client.deployments.create(model_uid, meta_props=meta_props)

deployment_uid = client.deployments.get_uid(deployment_details)

client.deployments.list()



#######################################################################################

Synchronous deployment creation for uid: 'c0a3f8e9-8831-49d4-af13-5c313495fe69' started

#######################################################################################


ready.


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='2d38acd4-7f85-410c-8197-38973cfb1a01'
------------------------------------------------------------------------------------------------


------------------------------------  ---------------  -----  ------------------------  -------------  ----------  ----------------
GUID                                  NAME             STATE  CREATED                   ARTIFACT_TYPE  SPEC_STATE  SPEC_REPLACEMENT
2d38acd4-7f85-410c-8197-38973cfb1a01  Supply Chain     ready  2023-08-01T15:36:03.985Z  model          supported
f639a3e1-9dce-46a5-ba40-9c7f7240497b  NurseDeployme

,GUID,NAME,STATE,CREATED,ARTIFACT_TYPE,SPEC_STATE,SPEC_REPLACEMENT
0,2d38acd4-7f85-410c-8197-38973cfb1a01,Supply Chain,ready,2023-08-01T15:36:03.985Z,model,supported,
1,f639a3e1-9dce-46a5-ba40-9c7f7240497b,NurseDeployment,ready,2023-06-30T03:05:28.082Z,model,,


In [15]:
deployment_uid

'2d38acd4-7f85-410c-8197-38973cfb1a01'

In [16]:
# Create another data and test with the model using a job

In [17]:
Plants = pd.DataFrame(["Plant1","Plant2"], columns = ["PlantName"])

Customers = pd.DataFrame(["Customer1","Customer2"], columns = ["CustomerName"])

Products = pd.DataFrame(["Product1","Product2"], columns = ["ProductName"])

ProductionCosts = pd.DataFrame([["Plant1","Product1",2],
                                ["Plant2","Product1",3],
                                ["Plant1","Product2",4],
                                ["Plant2","Product2",5]], columns = ["PlantName","ProductName","ProductionCost"]) 


Demand = pd.DataFrame([["Customer1","Product1",200],
                       ["Customer1","Product2",300],
                       ["Customer2","Product1",500],
                       ["Customer2","Product2",400]], columns = ["CustomerName","ProductName","Demand"])

TransportationCosts = pd.DataFrame([["Plant1","Customer1","Product1",0.2],
                                    ["Plant1","Customer1","Product2",0.5],
                                    ["Plant1","Customer2","Product1",0.3],
                                    ["Plant1","Customer2","Product2",0.4],
                                    ["Plant2","Customer1","Product1",0.25],
                                    ["Plant2","Customer1","Product2",0.2],
                                    ["Plant2","Customer2","Product1",0.35],
                                    ["Plant2","Customer2","Product2",0.3]], columns = ["PlantName","CustomerName","ProductName","TransportationCost"]) 

ProductionCapacity = pd.DataFrame([["Plant1","Product1",500],
                                   ["Plant2","Product1",400],
                                   ["Plant1","Product2",300],
                                   ["Plant2","Product2",600]], columns = ["PlantName","ProductName","Capacity"]) 


In [18]:
solve_payload = {
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA: [
        {
            "id":"Plants.csv",
            "values" : Plants
        },
        {
            "id":"Customers.csv",
            "values" : Customers
        },
        {
            "id":"Products.csv",
            "values" : Products
        },
        {
            "id":"ProductionCosts.csv",
            "values" : ProductionCosts
        },
        {
            "id":"Demand.csv",
            "values" : Demand
        },
        {
            "id":"TransportationCosts.csv",
            "values" : TransportationCosts
        },
        {
            "id":"ProductionCapacity.csv",
            "values" : ProductionCapacity
        }
    ],
    client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [
    {
        "id":".*\.csv"
    }
    ]
}
job_details = client.deployments.create_job(deployment_uid, solve_payload)
job_uid = client.deployments.get_job_uid(job_details)
job_uid

'a168a465-b8a9-4f58-8e59-bc795562d999'

In [19]:
from time import sleep

while job_details['entity']['decision_optimization']['status']['state'] not in ['completed', 'failed', 'canceled']:
    print(job_details['entity']['decision_optimization']['status']['state'] + '...')
    sleep(5)
    job_details=client.deployments.get_job_details(job_uid)

print( job_details['entity']['decision_optimization']['status']['state'])

queued...
queued...
queued...
queued...
running...
completed


In [20]:
print(client.deployments.get_job_details(job_uid)['entity']['decision_optimization']['status'])

{'completed_at': '2023-08-01T15:36:35.740Z', 'running_at': '2023-08-01T15:36:33.313Z', 'state': 'completed'}


In [21]:
for i in job_details['entity']['decision_optimization']['output_data']:
    print(i['id'])
    display(pd.DataFrame(i['values'], columns = i['fields']) )



TransportationPlan.csv


,X_transportation,X_transportation_Solution
0,X_transportation_Product1_Plant1_Customer1,0
1,X_transportation_Product1_Plant1_Customer2,500
2,X_transportation_Product1_Plant2_Customer1,200
3,X_transportation_Product1_Plant2_Customer2,0
4,X_transportation_Product2_Plant1_Customer1,0
5,X_transportation_Product2_Plant1_Customer2,300
6,X_transportation_Product2_Plant2_Customer1,300
7,X_transportation_Product2_Plant2_Customer2,100


Delivery.csv


,ProductName,CustomerName,Total_Delivered,Demand,Total_Delivered_Solution
0,Product1,Customer1,X_transportation_Product1_Plant1_Customer1+X_t...,200,200
1,Product1,Customer2,X_transportation_Product1_Plant1_Customer2+X_t...,500,500
2,Product2,Customer1,X_transportation_Product2_Plant1_Customer1+X_t...,300,300
3,Product2,Customer2,X_transportation_Product2_Plant1_Customer2+X_t...,400,400


ProductionPlan.csv


,ProductName,PlantName,X_production,Capacity,X_production_Solution
0,Product1,Plant1,X_production_Product1_Plant1,500,500
1,Product1,Plant2,X_production_Product1_Plant2,400,200
2,Product2,Plant1,X_production_Product2_Plant1,300,300
3,Product2,Plant2,X_production_Product2_Plant2,600,400


stats.csv


,Name,Value
0,cplex.modelType,LP
1,cplex.size.integerVariables,0
2,cplex.size.continousVariables,12
3,cplex.size.linearConstraints,12
4,cplex.size.booleanVariables,0
5,cplex.size.constraints,12
6,cplex.size.quadraticConstraints,0
7,cplex.size.variables,12
8,job.coresCount,1
9,job.inputsReadMs,1804


kpis.csv


,Name,Value
0,Total_Production_Costs,4800
1,Total_Transportation_Costs,410
2,Total_Costs,5210
